TCA & Glycolysis Flux

In [1]:
import pandas as pd
import pathlib
import incawrapper
from incawrapper import run_inca
import ast
PROJECT_DIR = pathlib.Path().cwd().parents[1].resolve()
data_folder = PROJECT_DIR / pathlib.Path("C:/Users/HP/Desktop/Master's Project/INCAWrapper")

c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandera\_pandas_deprecated.py:149: FutureWarning: Importing pandas-specific classes and functions from the
top-level pandera module will be **removed in a future version of pandera**.
If you're using pandera to validate pandas objects, we highly recommend updating
your import:

```
# old import
import pandera as pa

# new import
import pandera.pandas as pa
```

If you're using pandera to validate objects from other compatible libraries
like pyspark or polars, see the supported libraries section of the documentation
for more information on how to import pandera:

https://pandera.readthedocs.io/en/stable/supported_libraries.html

To disable this warning, set the environment variable:

```
export DISABLE_PANDERA_IMPORT_WARNING=True
```

  warnings.warn(_future_warning, FutureWarning)


In [2]:
tracers_data = pd.read_csv(data_folder / "tracers2.csv",
   converters={'atom_mdv':ast.literal_eval, 'atom_ids':ast.literal_eval} # a trick to read lists from csv
).query("experiment_id == 'exp1'") # we only simulated experiment 1

reactions_data = pd.read_csv(data_folder / "updated tca_gly model.csv")
ms_data = pd.read_csv(data_folder / "TCA_Glycolysis_ms file.csv",
  converters={'labelled_atom_ids': ast.literal_eval} # a trick to read lists from csv
)


In [3]:
output_file = PROJECT_DIR / pathlib.Path("C:/Users/HP/Desktop/Master's Project/INCAWrapper/TCA_Gly_flux_escher.mat")
script = incawrapper.create_inca_script_from_data(
    reactions_data=reactions_data,
    tracer_data=tracers_data,
    ms_measurements=ms_data,
    experiment_ids=["exp1"]
)
script.add_to_block("options",
    incawrapper.define_options(
        fit_starts=10, # Number of flux estimation restarts
        sim_na=False, # Do not simulate natural abundance
        sim_ss=False # Do INST 13C MFA
    )
)
script.add_to_block("runner", incawrapper.define_runner(output_file, run_estimate=True, run_continuation=True))

c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\incawrapper\core\INCAScript_writing.py:336: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ms_measurements[fill_columns] = ms_measurements[fill_columns].ffill().bfill()
c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\incawrapper\core\INCAScript_writing.py:336: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ms_measurements[fill_columns] = ms_measurements[fill_columns].ffill().bfill()
c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\incawrapper\co

In [4]:
import os

# Change working directory
os.chdir(r"C:/Users/HP/Desktop/Master's Project/INCAWrapper")

# Verify the change
print("Current working directory:", os.getcwd())


Current working directory: C:\Users\HP\Desktop\Master's Project\INCAWrapper


In [5]:
# Write the INCA script to a file
script_filename = "TCA_Gly_Flux_escher.m"
script.save_script(PROJECT_DIR / script_filename)
print(f"INCA script saved to {PROJECT_DIR / script_filename}.")

INCA script saved to C:\Users\HP\Desktop\TCA_Gly_Flux_escher.m.


Go to MATLAB:

cd 'C:/Program Files/INCA/inca2.4'
startup
setpath
inca_script

In [6]:
res = incawrapper.INCAResults(output_file)

In [7]:
res.fitdata.get_goodness_of_fit()

Fit accepted: False
Confidence level: 0.05
Chi-square value (SSR): 58738.512131954674
Expected chi-square range: [113.55571355 180.22912239]


In [8]:
estimated_fluxes = res.fitdata.fitted_parameters.query("type.str.contains('flux')")
estimated_fluxes

c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\incawrapper\core\INCAFitData.py:150: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  .apply(lambda x: pd.to_numeric(x, errors="ignore"))


,type,id,eqn,val,std,lb,ub,unit,free,alf,chi2s,cont,cor,cov,vals,base
0,Net flux,PGI net,G6P.c <-> F6P.c,0.000000,0.000000,0.0,0.0,NaN,0,0.05,"[58738.512131954674, 58738.512131954674]",0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0]",{'id': []}
1,Exch flux,PGI exch,G6P.c <-> F6P.c,0.000548,68842.387071,0.0,inf,NaN,1,0.05,"[58738.512131954674, 58738.512131954674, 58738...",0,"[0.0, 1.0000000000000002, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 4739274257.655172, 0.0, 0.0, 0.0, 0.0, 0...","[0.0, 0.0005478541559181605, 0.000547854155918...",{'id': []}
2,Net flux,PFK,F6P.c -> FBP.c + ADP,0.000000,0.000000,0.0,0.0,NaN,0,0.05,"[58738.512131954674, 58738.512131954674]",0,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0]",{'id': []}
3,Net flux,FBA net,FBP.c <-> DHAP.c + GAP.c,0.000000,0.000000,0.0,0.0,NaN,0,0.05,"[58738.512131954674, 58738.512131954674]",0,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0]",{'id': []}
4,Exch flux,FBA exch,FBP.c <-> DHAP.c + GAP.c,0.004722,68842.387071,0.0,inf,NaN,1,0.05,"[58738.512131954674, 58738.512131954674, 58738...",0,"[0.0, 0.0, 0.0, 0.0, 1.0000000000000002, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 4739274257.655172, 0.0, 0...","[0.0, 0.004722029690429662, 0.0047220296904296...",{'id': []}
5,Net flux,TPI net,DHAP.c <-> GAP.c,0.000000,0.000000,0.0,0.0,NaN,0,0.05,"[58738.512131954674, 58738.512131954674]",0,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0]",{'id': []}
6,Exch flux,TPI exch,DHAP.c <-> GAP.c,0.000456,68842.387071,0.0,inf,NaN,1,0.05,"[58738.512131954674, 58738.512131954674, 58738...",0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0000000000000...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4739274257.6551...","[0.0, 0.00045605912442781075, 0.00045605912442...",{'id': []}
7,Net flux,GAPD net,GAP.c <-> 3PG.c,0.000000,0.000000,0.0,0.0,NaN,0,0.05,"[58738.512131954674, 58738.512131954674]",0,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0]",{'id': []}
8,Exch flux,GAPD exch,GAP.c <-> 3PG.c,0.001237,68842.387071,0.0,inf,NaN,1,0.05,"[58738.512131954674, 58738.512131954674, 58738...",0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.000...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 47392...","[0.0, 0.0012372591955851432, 0.001237259195585...",{'id': []}
9,Net flux,PGK net,BPG.c <-> 3PG.c,0.000000,0.000000,0.0,0.0,NaN,0,0.05,"[58738.512131954674, 58738.512131954674]",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0]",{'id': []}


In [9]:
#Save the results to csv
estimated_fluxes.to_csv("estimated_fluxes.csv", index=False)


In [ ]:
import pandas as pd

# Read specific columns of csv file 
df = pd.read_csv("estimated_fluxes.csv", usecols=['id', 'val'])

# Remove 0 values
df_filtered = df[df['val'] != 0].copy()

# Remove 'exch' extension from the id names and strip whitespace
df_filtered['id'] = df_filtered['id'].str.replace(' exch', '').str.strip()

# Set 'id' column as the index
df_filtered.set_index('id', inplace=True)

# Select only the 'val' column as a Series, then save
df_filtered['val'].to_json('output.json', orient='index', indent=2)


           id          val
1         PGI 5.478542e-04
4         FBA     0.004722
6         TPI 4.560591e-04
8        GAPD     0.001237
10        PGK     0.001379
12        PGM 5.042221e-04
14        ENO     0.005753
17      LDH_L 3.181335e-04
22     ACONTm 6.716914e-04
24     ICDHxm     0.005386
27   SUCOAS1m     0.001355
29     SUCD1m     0.006477
31       FUMm 3.881965e-04
33       MDHm     0.002550


C:\Users\HP\AppData\Local\Temp\ipykernel_4564\2970905107.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['id'] = df_filtered['id'].str.replace(' exch', '')
C:\Users\HP\AppData\Local\Temp\ipykernel_4564\2970905107.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['id'] = df_filtered['id'].str.strip()  # Remove any leading/trailing spaces
